In [ ]:
aoi = 'abcd' # type: stage-in
columns = 'agbd, agbd_se, l2_quality_flag, l4_quality_flag, sensitivity, sensitivity_a2' # type: string
query = '' # type: string
limit = '10000' # type: int

In [ ]:
%%bash
# write a bash script here
/bin/bash maap-documentation-examples/gedi-subset/subset.sh {aoi} {columns} {query} {limit}

Couldn't find program: 'bash'
